In [2]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
from networkx.algorithms import bipartite
from tqdm import tqdm
#df = pd.read_excel('graph_測試.xlsx')
import pickle

In [3]:
df = pd.read_excel('all_patent.xlsx')
invt_ID = df['invt_ID'].unique()

In [4]:
import math
n = math.ceil(len(invt_ID))

In [5]:
df = df.dropna(how='all', axis=1)

In [6]:
#data is too big 拆成四個part跑(0,n),(n,2n),(2n,3n), (3n,len)
'''
inventor = []

for i in tqdm(range(len(invt_ID))):
    nodes_PN = []
    nodes_assignee = []
    edges = []
    for PN in df.loc[(df['invt_ID']==invt_ID[i])]['PN']:
        if PN not in nodes_PN:
            nodes_PN.append(PN)
    for ass_ID in df.loc[(df['invt_ID']== invt_ID[i])]['assignee_ID']:
        if ass_ID not in nodes_assignee:
            nodes_assignee.append(ass_ID)        
    for j in df.loc[(df['invt_ID']== invt_ID[i])].index:
        edges.append((df.loc[(df['invt_ID']==invt_ID[i])]['PN'][j],df.loc[(df['invt_ID']==invt_ID[i])]['assignee_ID'][j]))
    inventor.append({invt_ID[i]:{'nodes_PN':nodes_PN , 'nodes_assignee':nodes_assignee, 'edges':edges}})
'''

"\ninventor = []\n\nfor i in tqdm(range(len(invt_ID))):\n    nodes_PN = []\n    nodes_assignee = []\n    edges = []\n    for PN in df.loc[(df['invt_ID']==invt_ID[i])]['PN']:\n        if PN not in nodes_PN:\n            nodes_PN.append(PN)\n    for ass_ID in df.loc[(df['invt_ID']== invt_ID[i])]['assignee_ID']:\n        if ass_ID not in nodes_assignee:\n            nodes_assignee.append(ass_ID)        \n    for j in df.loc[(df['invt_ID']== invt_ID[i])].index:\n        edges.append((df.loc[(df['invt_ID']==invt_ID[i])]['PN'][j],df.loc[(df['invt_ID']==invt_ID[i])]['assignee_ID'][j]))\n    inventor.append({invt_ID[i]:{'nodes_PN':nodes_PN , 'nodes_assignee':nodes_assignee, 'edges':edges}})\n"

In [13]:
with open('inventor_part1.pickle', 'rb') as handle:
    a = pickle.load(handle)    
with open('inventor_part2.pickle', 'rb') as handle:
    b = pickle.load(handle)    
with open('inventor_part3.pickle', 'rb') as handle:
    c = pickle.load(handle)    
with open('inventor_part4.pickle', 'rb') as handle:
    d = pickle.load(handle)    

In [14]:
for i in range(len(b)):
    if i < 10:
        print(b[i])

{'fl:f_ln:krammer-1': {'nodes_PN': [9371366, 9968670, 10131695, 10137189, 10583188, 10736956], 'nodes_assignee': ['be5208ba-698e-4484-ae5d-dee9987f5b87'], 'edges': [(9371366, 'be5208ba-698e-4484-ae5d-dee9987f5b87'), (9968670, 'be5208ba-698e-4484-ae5d-dee9987f5b87'), (10131695, 'be5208ba-698e-4484-ae5d-dee9987f5b87'), (10137189, 'be5208ba-698e-4484-ae5d-dee9987f5b87'), (10583188, 'be5208ba-698e-4484-ae5d-dee9987f5b87'), (10736956, 'be5208ba-698e-4484-ae5d-dee9987f5b87')]}}
{'fl:f_ln:kratz-2': {'nodes_PN': [7902144, 8153581, 8664181, 8703724, 8778914, 8846602, 9216228, 9320803, 9446138, 9801949, 9901644, 10220101, 10426841], 'nodes_assignee': ['19721e36-cbfc-48b8-9a99-5cbf11b13940', '659220fb-fe47-4eeb-8bf5-b3a02e62f77a', '5ee1ddd9-a07a-4269-b560-38ce6029a7a0'], 'edges': [(7902144, '19721e36-cbfc-48b8-9a99-5cbf11b13940'), (8153581, '659220fb-fe47-4eeb-8bf5-b3a02e62f77a'), (8664181, '19721e36-cbfc-48b8-9a99-5cbf11b13940'), (8703724, '659220fb-fe47-4eeb-8bf5-b3a02e62f77a'), (8778914, '1972

In [15]:
inventor = a + b + c + d

In [16]:
len(inventor)

125267

In [101]:
#nx.draw_networkx(B, with_labels=True)

In [17]:
from networkx.algorithms import bipartite
def get_connected_len(nodes_PN, nodes_assignee, edges):
    B = nx.Graph()
    B.add_nodes_from(nodes_PN,bipartite=0)
    B.add_nodes_from(nodes_assignee,bipartite=1)
    B.add_edges_from(edges)
    bipartite.is_bipartite(B)
    edges = B.edges()
    #nx.draw_networkx(B,pos = nx.drawing.layout.bipartite_layout(B, nodes_PN), width = 2)
    return len(list(nx.connected_components(B)))

In [18]:
disconnected = []
for i in range(len(invt_ID)):
    for key, graph in inventor[i].items():
        node_pn = graph['nodes_PN']
        node_ass = graph['nodes_assignee']
        edge = graph['edges']
        if get_connected_len(node_pn, node_ass, edge) > 1:
            #print(key)
            disconnected.append(key)

In [19]:
print(len(disconnected))
#len(disconnected)

26605


In [20]:
disconnected

['10nq3ifgy58wnprplh3b1ea96',
 '1mzggiih6sdfrdzanlcyek6zr',
 '1z8jlgwvr64f3lolv6lklpy8d',
 '27qyqe9jlkpuhldak0qkrrcz8',
 '2lsx21kzqawn1hj4dpoqb6vtq',
 '2xrxzt9bccg109a97srqp3q0b',
 '3pqd8m73kr1j0dt3ttlnpyl9t',
 '4kusheign894lm8hb792336p2',
 '6kdb5jch6oycy3cbyfcm48hen',
 '6qs39ye53r6ab6dey557x7jwf',
 '7691rl421l6jujouto387ntjb',
 '7j1tdezm3jx6cqo5n5z0b4e4f',
 '8atd5abkzj6ck2tozpurgc70y',
 '8d8dth603an7ongvudhh1pjr1',
 '8i68zfkwyujdu9fp8s1vks07w',
 '8vqu411ticw03uan38j4npwuw',
 '9t2sccefsa8e0uzoomffzxtze',
 'aazyduxsm3da9zutitce8chly',
 'anio9w2au1kvf2dyhfbrajyue',
 'as50yh33qz6svydcjprth5p65',
 'bzkdjl7sc84efae86em2i7hgr',
 'e835hfycewmezq0y8otmqffqy',
 'fewlpfmipshcus4p433f823d4',
 'fl:a_ln:abdelgader-1',
 'fl:a_ln:abouabdellah-1',
 'fl:a_ln:abraham-9',
 'fl:a_ln:abram-1',
 'fl:a_ln:acharya-2',
 'fl:a_ln:ackerman-4',
 'fl:a_ln:adamis-1',
 'fl:a_ln:adejare-1',
 'fl:a_ln:adhikari-2',
 'fl:a_ln:aerts-2',
 'fl:a_ln:agarwal-2',
 'fl:a_ln:agarwal-6',
 'fl:a_ln:agrawal-1',
 'fl:a_ln:aguilar-2